# **Project 1: Linear Panel Data and Production Technology**

By Emma Knippel, Anna Abildskov and Oscar Nyholm

**Table of contents**
* [Setup](#toc0_)

* [Introduction](#toc1_)   

* [Read and clean data](#toc2_) 

* [FE estimation of $\beta_K$ and $\beta_L$](#toc3_)    

* [Test for constant returns to scale](#toc4_)    

* [Conclusion](#toc5_)   

## <a id='toc0_'><a>[Setup](#toc0_)

In [8]:
import numpy as np
from numpy import linalg as la
import pandas as pd
from io import StringIO
from tabulate import tabulate
from matplotlib import pyplot as plt

#Supress Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Import our py-file
import Project_1 as pf

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <a id='toc1_'></a>[Introduction](#toc1_)

- intro
- intro
- intro


We are estimating parameters of a Cobb-Douglas production function of the form:
$$\begin{aligned}
y_it = \beta_kk_{it}+\beta_Ll_{it}+v_{it},
\end{aligned}$$
Where $v_{it} =\ln A_{it}$.

## <a id='toc2_'></a>[Read and clean data](#toc2_)

The dataset contains `N = 441` firms observed over `T = 12` years, 1968-1979. There variables are: 
* `lcap`: Log of capital stock, $k_{it}$ 
* `lemp`: log of employment, $\ell_{it}$ 
* `ldsa`: log of deflated sales, $y_{it}$
* `year`: the calendar year of the observation, `year` $ = 1968, ..., 1979$, 
* `firmid`: anonymized indicator variable for the firm, $i = 1, ..., N$, with $N=441$. 

Loading the data and dropping all the even years, leaving us with only odd year-observations:

In [9]:
df_all_years = pd.read_csv('firms.csv')
data = df_all_years[df_all_years.year % 2 != 0]

Converting data to numpy format:

In [10]:
N = data.firmid.unique().size
T = data.year.unique().size
print(f'Data contains data for {N} firms over {T} odd years')

Data contains data for 441 firms over 6 odd years


Extracting data to numpy arrays:

In [11]:
y = data.ldsa.values.reshape((N*T,1))
const = np.ones((N*T,1))
l = data.lemp.values.reshape((N*T,1))
k = data.lcap.values.reshape((N*T,1))
X = np.hstack([const, l, k])

## <a id='toc3_'></a>[FE estimation of $\beta_K$ and $\beta_L$](#toc3_)

## <a id='toc4_'></a>[Test for constant returns to scale](#toc4_)

## <a id='toc5_'></a>[Conclusion](#toc5_)